# Imports

In [1]:
from dqn_agent import DQNAgent
from ppo_agent import PPOAgent
from data_processing import load_preprocessed_dataset, get_biased_feature_percentage, train_test_split_data, get_xy_from_data, get_activity, get_column_units
from visualize import visualize_results, visualize_training, visualize_preprocessed_data
import numpy as np
from sklearn.model_selection import train_test_split

# Load Data

In [2]:
# Load the preprocessed dataset
df = load_preprocessed_dataset(verbose=True, drop_subject_id=True)

# Get percentage of biased feature
biased_feature_percentage = get_biased_feature_percentage(df, print_result=False)

# Split data into 80% training and 20% testing
train_df, test_df, seed = train_test_split_data(df, split_size=0.2, random_state=None)

# Get X and y data from training and testing sets
x_train, y_train = get_xy_from_data(train_df, target_feature_name="Activity ID")
x_test, y_test = get_xy_from_data(test_df, target_feature_name="Activity ID")

print("--- Dataset Preperation Complete ---")
print(f"{y_train.shape[0]} training samples")
print(f"{y_test.shape[0]} testing samples")
print(f"Random state seed used for train/test split: {seed}")
print(f"Percentage of rows where 'Sex - Female' == 1 is ~{biased_feature_percentage*100:.2f}%")
print("------------------------------------")

---- Preprocessed Dataset Info ----
Number of rows: 61000
Number of columns: 40
-----------------------------------
--- Dataset Preperation Complete ---
48800 training samples
12200 testing samples
Random state seed used for train/test split: 4200360045
Percentage of rows where 'Sex - Female' == 1 is ~14.75%
------------------------------------


# Initialize system

In [3]:
dqn_config = {
    'state_size': df.shape[1] - 1,  
    'action_size': 2,  
    'hidden_size': 64,
    'lr': 1e-3,
    'gamma': 0.99,
    'batch_size': 32,
    'memory_size': 10000,
    'epsilon_start': 1.0,
    'epsilon_min': 0.01,
    'epsilon_decay': 0.995
}

ppo_config = {
    'state_size': df.shape[1] - 1,  
    'action_size': 2,  
    'hidden_size': 64,
    'lr': 3e-4,
    'gamma': 0.99,
    'clip_epsilon': 0.2,
    'update_epochs': 4,
    'batch_size': 64,
    'c1': 0.5,
    'c2': 0.01
}

dqn_agent = DQNAgent(**dqn_config)
ppo_agent = PPOAgent(**ppo_config)

# Training

In [4]:
def train_agent(agent, df, agent_name):
    if isinstance(agent, DQNAgent):
        for _ in range(5):
            for index, row in df.iterrows():
                state = row[:-1].values
                action = agent.predict(state)
                reward = row[-1]  # Assuming reward is last column
                next_state = state  # DQNAgent does not generate next state; add logic if needed
                done = False
                agent.remember(state, action, reward, next_state, done)
                agent.train()
    else:  # PPO
        for episode in range(5):
            total_reward = 0
            for index, row in df.iterrows():
                state = row[:-1].values
                action = agent.predict(state)
                reward = row[-1]  # Assuming reward is last column
                done = False
                agent.store_transition(state, action, agent.last_log_prob, reward, done, agent.last_value)
            agent.train()

print("Training DQN Agent...")
dqn_agent.save("dqn_trained_model.pth")

print("Training PPO Agent...")
ppo_agent.save("ppo_trained_model.pth")

Training DQN Agent...
Training PPO Agent...


# Testing


In [5]:
def test_agent(agent, df, agent_name):
    total_reward = 0
    for index, row in df.iterrows():
        state = row[:-1].values
        total_reward += agent.predict(state)
    print(f"{agent_name} Test Reward: {total_reward}")

print("Testing DQN Agent...")
dqn_agent.save("dqn_trained_model.pth")
test_agent(dqn_agent, df, "DQN")

print("Testing PPO Agent...")
ppo_agent.save("ppo_trained_model.pth")
test_agent(ppo_agent, df, "PPO")

Testing DQN Agent...
DQN Test Reward: 87938
Testing PPO Agent...
PPO Test Reward: 170499


# Visualize results

In [6]:
# visualize_training([], [])
# visualize_results(dqn_agent, ppo_agent, df)
# visualize_preprocessed_data(df)